In [4]:
import numpy as np
import pandas as pd
import dask.dataframe as ddf

In [13]:
c = ddf.read_csv('/Volumes/Expansion/traffic-prediction/data/its-소통/1/its_c_1_1_m1.csv', dtype={'링크ID':'object'})
e = ddf.read_csv('/Volumes/Expansion/traffic-prediction/data/its-돌발/1/its_e_1_1_m1.csv', dtype={'링크아이디':'object'})
w = ddf.read_csv('/Volumes/Expansion/traffic-prediction/data/기상청-방재/weather_1_m1.csv')
r = ddf.read_csv('/Volumes/Expansion/traffic-prediction/data/도로 데이터/road(info)_m1.csv')
area = ddf.read_csv(
    '/Volumes/Expansion/traffic-prediction/data/전국교통사고다발지역표준데이터/전국교통사고다발지역표준데이터.csv')
cam = ddf.read_csv(
    '/Volumes/Expansion/traffic-prediction/data/전국무인교통단속카메라표준데이터/전국무인교통단속카메라표준데이터_m1.csv')
node = ddf.read_csv('/Volumes/Expansion/traffic-prediction/data/표준노드링크/data/node_m1.csv')
link = ddf.read_csv('/Volumes/Expansion/traffic-prediction/data/표준노드링크/data/link_m1.csv')
men = ddf.read_csv(
    '/Volumes/Expansion/traffic-prediction/data/행정안전부_지역별(행정동) 성별 연령별 주민등록 인구수/행정안전부_지역별(행정동) 성별 연령별 주민등록 인구수_20240731.csv')
con = ddf.read_csv('/Volumes/Expansion/traffic-prediction/data/혼잡빈도/confusion_1_m1.csv')

In [6]:
# 데이터 병합 순선
# 1. 소통 데이터 데이트 컬럼 생성
# 2. 돌발 데이터 데이트 컬럼 생성(반올림 함수)
# 3. 둘 결합
# 4. 링크 데이터 링크 아이디 개명 후(영어로 통일) 결합(위도 경도는 중점으로.)
# 5. 노드 아이디도 결합
# 6. 날씨 데이터와 날씨 지점 데이터 결합
# 7. 기존 결합데이터에서 가장 유클리디안 거리로 가까운 지점명 컬럼 생성
# 8. 결합 날씨 데이터와 결합
# 9. 혼잡빈도 데이터 결합
# 10. 그외 잡 변수 결합
# 11. 링크 별로 그룹화, 중복 제거 후 배치로 내보내기

In [7]:
    def get_big_date(v):
        a = str(v)
        return a[:4] + '-' + a[4:6] + '-' + a[6:8]

    def get_time(v):
        a = str(v).rjust(4, '0')
        return a[:2] + ':' + a[2:]

    def remove_s(v):
        a = str(v)
        return a[:-2] + '00'

    def custom_round(a):
        s = str(a)
        number = int(s[14:16])

        if number >= 53:
            return s
        else:
            remainder = number % 10  # 1의 자리 숫자를 얻기 위해 10으로 나눈 나머지로 한다.
        result = 0
        if remainder in [0, 1, 2]:
            result = number - remainder  # 0, 1, 2는 0으로 반올림
        elif remainder in [3, 4, 5, 6, 7]:
            result = number - remainder + 5  # 3, 4, 5, 6, 7은 5로 반올림
        else:
            result = number - remainder + 10  # 8, 9는 0으로 반올림

        return s[:14] + str(result) + s[16:]

In [8]:
c_data = c.compute()

# 컬럼 이름 변경
c_data = c_data.rename(columns={'링크ID':'LINK_ID'})
c_data = c_data.astype({'LINK_ID':'str'})

In [9]:
c_data['date'] = pd.to_datetime(c_data['생성일'].apply(get_big_date) + ' ' + c_data['생성시분'].apply(get_time))
c_data = c_data[pd.Index([c_data.columns[-1]]).append(c_data.columns[:-1])]

In [12]:
c_data.tail()

,date,생성일,생성시분,LINK_ID,통행속도,통행시간
2025573,2023-01-01 23:55:00,20230101,2355,4180383901,4.0,0
2025574,2023-01-01 23:55:00,20230101,2355,4180384001,97.0,0
2025575,2023-01-01 23:55:00,20230101,2355,4180384101,76.0,0
2025576,2023-01-01 23:55:00,20230101,2355,4180384201,115.0,0
2025577,2023-01-01 23:55:00,20230101,2355,4180384301,79.0,0


In [ ]:
e_data = e.compute()